___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Question and Answer Chat Bots

----

------

## Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [1]:
import pickle
import numpy as np

In [2]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [3]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

----

## Exploring the Format of the Data

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(test_data)

1000

In [7]:
len(train_data)

10000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [10]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [11]:
train_data[0][2]

'no'

-----

## Setting up Vocabulary of All Words

In [12]:
# Create a set that holds the vocab words
vocab = set()

In [13]:
all_data = test_data + train_data

In [14]:
for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [15]:
vocab.add('no')
vocab.add('yes')

In [16]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [17]:
vocab_len = len(vocab) + 1 #we add an extra space to hold a 0 for Keras's pad_sequences

In [18]:
max_story_len = max([len(data[0]) for data in all_data])

In [19]:
max_story_len

156

In [20]:
max_question_len = max([len(data[1]) for data in all_data])

In [21]:
max_question_len

6

## Vectorizing the Data

In [22]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [23]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

-----------

In [24]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [25]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [26]:
tokenizer.word_index

{'garden': 1,
 'travelled': 2,
 'milk': 3,
 'went': 4,
 'sandra': 5,
 'kitchen': 6,
 'yes': 7,
 'down': 8,
 '.': 9,
 'daniel': 10,
 'took': 11,
 'bedroom': 12,
 'moved': 13,
 'office': 14,
 'left': 15,
 'got': 16,
 'the': 17,
 'to': 18,
 'up': 19,
 'apple': 20,
 'football': 21,
 'bathroom': 22,
 'is': 23,
 'grabbed': 24,
 'there': 25,
 'john': 26,
 'discarded': 27,
 'no': 28,
 'picked': 29,
 'hallway': 30,
 'put': 31,
 'dropped': 32,
 'in': 33,
 'mary': 34,
 'journeyed': 35,
 '?': 36,
 'back': 37}

In [27]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [28]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [29]:
len(train_story_text)

10000

In [30]:
len(train_story_seq)

10000

In [31]:
# word_index = tokenizer.word_index

### Functionalize Vectorization

In [32]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [33]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [34]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [35]:
inputs_test

array([[ 0,  0,  0, ..., 17, 12,  9],
       [ 0,  0,  0, ..., 17,  1,  9],
       [ 0,  0,  0, ..., 17,  1,  9],
       ...,
       [ 0,  0,  0, ..., 17, 20,  9],
       [ 0,  0,  0, ..., 17,  1,  9],
       [ 0,  0,  0, ..., 20, 25,  9]])

In [36]:
queries_test

array([[23, 26, 33, 17,  6, 36],
       [23, 26, 33, 17,  6, 36],
       [23, 26, 33, 17,  1, 36],
       ...,
       [23, 34, 33, 17, 12, 36],
       [23,  5, 33, 17,  1, 36],
       [23, 34, 33, 17,  1, 36]])

In [37]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [39]:
tokenizer.word_index['yes']

7

In [40]:
tokenizer.word_index['no']

28

## Creating the Model

In [41]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

### Placeholders for Inputs

Recall we technically have two inputs, stories and questions. So we need to use placeholders. `Input()` is used to instantiate a Keras tensor.


In [42]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

### Building the Networks

To understand why we chose this setup, make sure to read the paper we are using:

* Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895

## Encoders

### Input Encoder m

In [43]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### Input Encoder c

In [44]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

### Question Encoder

In [45]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

### Encode the Sequences

In [46]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

##### Use dot product to compute the match between first input vector seq and the query

In [47]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

#### Add this match matrix with the second input vector sequence

In [48]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

#### Concatenate

In [49]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [50]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [51]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)

In [52]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [53]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 64)     2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
______________________________________________________________________________________________

In [55]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
313/313 [==============================] - 4s 8ms/step - loss: 0.9250 - accuracy: 0.4940 - val_loss: 0.6944 - val_accuracy: 0.5030
Epoch 2/120
313/313 [==============================] - 2s 7ms/step - loss: 0.7051 - accuracy: 0.4965 - val_loss: 0.6953 - val_accuracy: 0.4970
Epoch 3/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6967 - accuracy: 0.4920 - val_loss: 0.6946 - val_accuracy: 0.4970
Epoch 4/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6951 - accuracy: 0.4918 - val_loss: 0.6933 - val_accuracy: 0.4970
Epoch 5/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6947 - accuracy: 0.5010 - val_loss: 0.6938 - val_accuracy: 0.4970
Epoch 6/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6946 - accuracy: 0.4956 - val_loss: 0.6934 - val_accuracy: 0.5030
Epoch 7/120
313/313 [==============================] - 2s 7ms/step - loss: 0.6948 - accuracy: 0.5003 - val_loss: 0.6943 - val_accuracy: 0.4970

Epoch 58/120
313/313 [==============================] - 2s 6ms/step - loss: 0.3285 - accuracy: 0.8594 - val_loss: 0.4239 - val_accuracy: 0.8180
Epoch 59/120
313/313 [==============================] - 2s 7ms/step - loss: 0.3270 - accuracy: 0.8594 - val_loss: 0.4450 - val_accuracy: 0.8110
Epoch 60/120
313/313 [==============================] - 2s 7ms/step - loss: 0.3304 - accuracy: 0.8559 - val_loss: 0.3988 - val_accuracy: 0.8260
Epoch 61/120
313/313 [==============================] - 2s 6ms/step - loss: 0.3231 - accuracy: 0.8610 - val_loss: 0.4409 - val_accuracy: 0.8140
Epoch 62/120
313/313 [==============================] - 2s 6ms/step - loss: 0.3198 - accuracy: 0.8628 - val_loss: 0.4315 - val_accuracy: 0.8130
Epoch 63/120
313/313 [==============================] - 2s 6ms/step - loss: 0.3185 - accuracy: 0.8610 - val_loss: 0.4463 - val_accuracy: 0.8090
Epoch 64/120
313/313 [==============================] - 2s 6ms/step - loss: 0.3149 - accuracy: 0.8659 - val_loss: 0.4018 - val_accuracy:

Epoch 115/120
313/313 [==============================] - 2s 6ms/step - loss: 0.2478 - accuracy: 0.8985 - val_loss: 0.4639 - val_accuracy: 0.8350
Epoch 116/120
313/313 [==============================] - 2s 7ms/step - loss: 0.2440 - accuracy: 0.9012 - val_loss: 0.4804 - val_accuracy: 0.8130
Epoch 117/120
313/313 [==============================] - 2s 7ms/step - loss: 0.2389 - accuracy: 0.8997 - val_loss: 0.4879 - val_accuracy: 0.8250
Epoch 118/120
313/313 [==============================] - 2s 7ms/step - loss: 0.2357 - accuracy: 0.9042 - val_loss: 0.4836 - val_accuracy: 0.8210
Epoch 119/120
313/313 [==============================] - 2s 6ms/step - loss: 0.2413 - accuracy: 0.9027 - val_loss: 0.4792 - val_accuracy: 0.8220
Epoch 120/120
313/313 [==============================] - 2s 7ms/step - loss: 0.2426 - accuracy: 0.8987 - val_loss: 0.4784 - val_accuracy: 0.8170


### Saving the Model

In [56]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

C:\Users\maira\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


## Evaluating the Model

### Plotting Out Training History

In [57]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


KeyError: 'acc'

### Evaluating on Given Test Set

In [58]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

In [59]:
test_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [60]:
story =' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [61]:
query = ' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [62]:
print("True Test Answer from Data is:",test_data[0][2])

True Test Answer from Data is: no


In [78]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.9574628


## Writing Your Own Stories and Questions

Remember you can only use words from the existing vocab

In [64]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [79]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [80]:
my_question = "Is the football in the garden ?"
#my_question = "john has football ?"

In [81]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [82]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [83]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [84]:
pred_results = model.predict(([ my_story, my_ques]))

In [85]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.9574628


# Great Job!